In [1468]:
import pandas as pd
import numpy as np

In [1469]:
df = pd.read_csv('..\\Bases\\Criados\\base_doencas_cardiacas.csv')

In [1470]:
df.columns[df.isnull().any()]

Index(['D00901', 'D014', 'E006011', 'E008', 'E01602', 'E017', 'E01802', 'E019',
       'F001021', 'F007021', 'F008021', 'VDF00102', 'G060', 'G062', 'I006',
       'I004', 'I00401', 'I00402', 'I00403', 'J00402', 'J00404', 'J00801',
       'M00203', 'M011011', 'M011021', 'M011051', 'M011071', 'N004', 'N005',
       'N008', 'P00104', 'P00404', 'P02801', 'P029', 'P035', 'P03701',
       'P03702', 'P051', 'P052', 'P053', 'P05401', 'P05404', 'P05407',
       'P05410', 'P05413', 'P05416', 'P05419', 'Q00201', 'Q03001', 'Q060',
       'Q064', 'Q06506', 'Q06507', 'Q06508', 'Q06509', 'Q06601', 'Q067',
       'H003', 'H004', 'H010', 'D00301', 'D008', 'E002', 'E003', 'E004',
       'E005', 'E011', 'E014011', 'E01601', 'E01603', 'E01605', 'E01801',
       'I00103', 'I001031', 'P00103', 'P00403', 'P05901', 'P05902', 'P05903',
       'P05904', 'Q06307', 'Q06308', 'Q06309', 'Q06310', 'Q06311', 'H002'],
      dtype='object')

In [1471]:
(df.isnull().sum() > 0).sum()

np.int64(86)

### Escolaridade

Atributos utilizados:
1. D00201: se frequenta escola atualmente
2. D00301: qual curso frequenta
3. D008: se já frequentou escola ou creche
4. D00901: Qual foi o curso mais elevado que frequentou?
5. D014: se concluiu esse curso

In [1472]:
def categorizar_escolaridade(row):
    if row['D00201'] == 1:
        return 'Frequenta'
    elif row['D00201'] == 2:
        if row['D008'] == 1:
            return 'Frequentou'
        elif row['D008'] == 2:
            return 'Não frequentou'
    return pd.NA  # Caso não se encaixe em nenhuma condição

df['Situacao_escolar'] = df.apply(categorizar_escolaridade, axis=1)

print(df['Situacao_escolar'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Situacao_escolar'].isna().sum())


Situacao_escolar
Frequentou        20923
Não frequentou     2741
Frequenta           687
Name: count, dtype: int64
Valores faltantes: 0


In [1473]:
df['D00301'].value_counts()

D00301
8.0     214
4.0     115
9.0     114
5.0      96
3.0      54
6.0      44
7.0      22
10.0     19
11.0      9
Name: count, dtype: int64

In [1474]:
def map_escolaridade(row): 
    situacao = row['Situacao_escolar']
    curso = row['D00901']
    concluiu = row['D014']
    curso_atual = row['D00301']

    # Se nunca frequentou escola, categoria = 0
    if situacao == 'Não frequentou':
        return 0

    # Usa D00901 (curso mais elevado) prioritariamente
    if curso in [1, 2]:  # Creche, Pré-escola
        categoria = 1
    elif curso in [3, 4, 5, 6, 7, 8]:  # Alfabetização, primário, EJA fundamental etc.
        categoria = 2
    elif curso in [9, 10, 11]:  # Ensino médio, EJA médio
        categoria = 3
    elif curso == 12:  # Superior
        categoria = 4
    elif curso in [13, 14, 15]:  # Pós-graduação
        categoria = 5
    # Se curso é inválido ou ausente, tenta usar curso_atual (D00301)
    elif curso_atual in [3, 4, 5]:
        categoria = 1  # Educação infantil
    elif curso_atual in [6, 7]:
        categoria = 2  # Fundamental
    elif curso_atual in [8, 9]:
        categoria = 3  # Médio
    elif curso_atual in [10, 11]:
        categoria = 4  # Superior
    else:
        return np.nan

    # Rebaixar se não concluiu (exceto se usou curso_atual)
    if concluiu == 2 and curso in range(1, 16):
        categoria -= 1
        if categoria < 0:
            categoria = 0

    return categoria



df['Escolaridade'] = df.apply(map_escolaridade, axis=1)

print(df['Escolaridade'].value_counts(dropna=False))

# 0: Nunca frequentou escola
# 1: Educação infantil
# 2: Ensino fundamental
# 3: Ensino médio
# 4: Ensino superior
# 5: Pós-graduação

Escolaridade
1    9443
3    5266
2    4133
0    2741
4    1828
5     940
Name: count, dtype: int64


In [1475]:
df.drop(columns=['D00201', 'D00301', 'D008', 'D00901', 'D014', 'Situacao_escolar'], inplace=True)

### Renda

1. E014011: se era trabalhador não remunerado ajudando parente/domicílio
2. E01601: se recebia dinheiro no trabalho principal
3. E01602: valor da renda do trabalho principal
4. E01801: se recebia dinheiro em outro(s) trabalho(s)
5. E01802: valor da renda dos outros trabalhos

Regras:

1. Se E01601 != 1 ou E01801 != 1, a renda respectiva é 0;
2. Se a pessoa é não remunerada (E014011 = 1, 2, 3), renda é 0;
3. Se E01602 ou E01802 estiverem NaN, mas a pessoa disse que recebia (E01601 ou E01801 = 1), considerar como NaN final.
4. Renda total é a soma de E01602 e E01802.

In [1476]:
def calcular_renda(row):
    # Regra 2: trabalhador não remunerado (1, 2 ou 3)
    if row['E014011'] in [1, 2, 3]:
        return 0.0

    # Regra 1 + 3: verificar se recebia dinheiro
    if row['E01601'] == 1:
        renda1 = row['E01602'] if pd.notna(row['E01602']) else np.nan
    else:
        renda1 = 0.0

    if row['E01801'] == 1:
        renda2 = row['E01802'] if pd.notna(row['E01802']) else np.nan
    else:
        renda2 = 0.0

    # Regra 4: soma ajustada
    if pd.isna(renda1) or pd.isna(renda2):
        if pd.notna(renda1) and pd.isna(renda2):
            return renda1
        elif pd.notna(renda2) and pd.isna(renda1):
            return renda2
        else:
            return np.nan

    return renda1 + renda2

df['Renda'] = df.apply(calcular_renda, axis=1)
print(f"Valores faltantes:",df['Renda'].isna().sum())

Valores faltantes: 0


In [1477]:
print((df['Renda'] == 0.0).sum())

12208


In [1478]:
df.drop(columns=['E014011', 'E01601', 'E01602', 'E01801', 'E01802'], inplace=True)

### Horas trabalhadas *incompleto (checar quant trabalho 3. Três ou mais)

1. E011: número de trabalhos na semana 
2. E017: horas trabalhas em 1 trabalho
3. E019: horas trabalhas em mais de 1 trabalho

In [1479]:
def quantidade_trabalhos(row):
    if row['E011'] == 1:
        return 1
    elif row['E011'] == 2:
            return 2
    elif row['E011'] == 3:
            return 3
    else:
         return 0

df['Quant_trabalhos'] = df.apply(quantidade_trabalhos, axis=1)

print(df['Quant_trabalhos'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Quant_trabalhos'].isna().sum())

Quant_trabalhos
0    11977
1    11634
2      655
3       85
Name: count, dtype: int64
Valores faltantes: 0


In [1480]:
print("E017 faltantes:", df['E017'].isna().sum())
print("E019 faltantes:", df['E019'].isna().sum())

E017 faltantes: 11977
E019 faltantes: 23611


In [1481]:
df['Quant_trabalhos'] = df['Quant_trabalhos'].astype(int)
df.loc[df['Quant_trabalhos'] == 0, 'E017'] = 0
df.loc[df['Quant_trabalhos'].isin([0, 1]), 'E019'] = 0

#df['E017']=df['E017'].fillna(0)
#df['E019']=df['E019'].fillna(0)

In [1482]:
print("E017 faltantes:", df['E017'].isna().sum())
print("E019 faltantes:", df['E019'].isna().sum())

E017 faltantes: 0
E019 faltantes: 0


In [1483]:
df['Horas_trabalhadas'] = df['E017'] + df['E019']

print("Valores faltantes:", df['Horas_trabalhadas'].isna().sum())

Valores faltantes: 0


In [1484]:
df['Quant_trabalhos'] = pd.to_numeric(df['Quant_trabalhos'], errors='coerce')
df[(df['Renda'] == 0.0) & (df['Quant_trabalhos'] == 0) & (df['Horas_trabalhadas'] == 0.0)][['Renda', 'Quant_trabalhos', 'Horas_trabalhadas']]

,Renda,Quant_trabalhos,Horas_trabalhadas
0,0.0,0,0.0
4,0.0,0,0.0
6,0.0,0,0.0
9,0.0,0,0.0
10,0.0,0,0.0
...,...,...,...
24314,0.0,0,0.0
24325,0.0,0,0.0
24326,0.0,0,0.0
24334,0.0,0,0.0


In [1485]:
df.drop(columns=['E01603', 'E01605', 'E017', 'E019'], inplace=True)

### Afastamento do trabalho por motivo de saúde & motivo de saúde relacionado ao trabalho

1. E005: tinha algum trabalho remunerado do qual estava temporariamente afastado?
2. E006011: por que motivo estava afastado desse trabalho
3. E008: a doença ou acidente foi relacionado ao trabalho?

In [1486]:
def temporariamente_afastado(row):
    if row['E005'] == 1:
        return 'Sim'
    elif row['E005'] == 2:
            return 'Não'
    else:
         return 'Não'

df['Temporariamente_afastado'] = df.apply(temporariamente_afastado, axis=1)


print(df['Temporariamente_afastado'].value_counts(dropna=False))
print(f"Valores faltantes:",df['Temporariamente_afastado'].isna().sum())

Temporariamente_afastado
Não    24022
Sim      329
Name: count, dtype: int64
Valores faltantes: 0


In [1487]:
df.loc[(df['Temporariamente_afastado']== 'Não'), 'E006011'] = 0 #se E006011=0, não esteve afastado

In [1488]:
df['E006011'].isna().sum()

np.int64(0)

In [1489]:
df.loc[(df['E006011'].isin([0, 1, 2, 4, 5, 6, 7])), 'E008'] = 0

In [1490]:
df['E008'].isna().sum()

np.int64(0)

In [1491]:
pd.set_option('future.no_silent_downcasting', True)

df['Temporariamente_afastado'] = df['Temporariamente_afastado'].replace({
    'Sim': 1,
    'Não': 2
}).astype(float)

In [1492]:
df.drop(columns=['E001','E002', 'E003', 'E004', 'E005'], inplace=True)

### Mobilidade

1. G059: usa algum aparelho de auxílio para se locomover?
2. G060: faz uso de cadeira de rodas?
3. G062: faz uso de bengala, muletas ou andador?

In [1493]:
def classificar_mobilidade(row):
    usa_aparelho = row['G059'] == 1
    cadeira_rodas = row['G060'] == 1
    bengala_muleta = row['G062'] == 1

    if not usa_aparelho:
        return 0.0  # Não usa nenhum aparelho
    if cadeira_rodas:
        return 1.0  # Só cadeira de rodas
    if bengala_muleta:
        return 2.0  # Só bengala/muleta/andador
    if cadeira_rodas and bengala_muleta:
        return 3.0  # Usa os dois
    if not cadeira_rodas and not bengala_muleta:
        return 4.0  # Não usa nenhum dos dois (uso outras coisas)
    return np.nan  # Caso de inconsistência ou valores faltantes

df['Mobilidade_auxiliar'] = df.apply(classificar_mobilidade, axis=1)

# Visualizar resultado:
print(df['Mobilidade_auxiliar'].value_counts(dropna=False))
print(df['Mobilidade_auxiliar'].isna().sum())

Mobilidade_auxiliar
0.0    23631
2.0      517
1.0      149
4.0       54
Name: count, dtype: int64
0


In [1494]:
df = df.drop(columns=['G060', 'G062'])

### Plano de Saúde

1. I00102: se tem plano
2. I006: avaliação do plano

In [1495]:
def classificar_plano(row):
    tem_plano = row['I00102']
    avaliacao = row['I006']
    
    if tem_plano == 2:
        return 'Não tem plano'  # Não tem plano
    elif avaliacao in [1, 2]:
        return 'Bom/muito bom'  # Muito bom ou bom
    elif avaliacao == 3:
        return 'Regular'  # Regular
    elif avaliacao in [4, 5]:
        return 'Ruim/muito ruim' # Ruim ou muito ruim
    elif avaliacao == 6:
        return 'Nunca usou'  # Nunca usou
    else:
        return np.nan  # Valores faltantes ou inválidos

df['Avaliacao_plano'] = df.apply(classificar_plano, axis=1)

print(df['Avaliacao_plano'].value_counts(dropna=False))
print(df['Avaliacao_plano'].isna().sum())

# 0     | Não tem plano                
# 1     | Avaliação muito boa ou boa   
# 2     | Avaliação regular            
# 3     | Avaliação ruim ou muito ruim 
# 4     | Nunca usou o plano 

Avaliacao_plano
Não tem plano      19748
Bom/muito bom       3409
Regular              941
Ruim/muito ruim      157
Nunca usou            96
Name: count, dtype: int64
0


In [1496]:
df[['Avaliacao_plano']]

,Avaliacao_plano
0,Não tem plano
1,Não tem plano
2,Regular
3,Regular
4,Não tem plano
...,...
24346,Não tem plano
24347,Não tem plano
24348,Não tem plano
24349,Não tem plano


In [1497]:
# Mapeamento explícito
mapeamento = {
    "Não tem plano": 0,
    "Bom/muito bom": 1,
    "Regular": 2,
    "Ruim/muito ruim": 3,
    "Nunca usou": 4
}

# Aplicando o mapeamento
df["Avaliacao_plano"] = df["Avaliacao_plano"].replace(mapeamento).astype(int)

In [1498]:
df[["Avaliacao_plano"]]

,Avaliacao_plano
0,0
1,0
2,2
3,2
4,0
...,...
24346,0
24347,0
24348,0
24349,0


In [1499]:
df = df.drop(columns=['I006'])

3. I004: se é de instituição pública
4. I00103: se é morador 
5. I001031: número do titular

*C00301 == I001031 --> é titular    

In [1500]:
def definir_plano_publico(row):
    if row['I00102'] != 1:
        return 0.0  # Não tem plano
    if row['I00103'] != 1 and row['C00301'] != row['I001031']: # não é morador
        return 0.0  # não é titular do plano
    if row['I004'] == 1:
        return 1.0  # Plano é de instituição pública
    if row['I004'] == 2:
        return 2.0  # Plano não é de instituição pública
    
    return 0.0

df['Plano_publico'] = df.apply(definir_plano_publico, axis=1)

print(df['Plano_publico'].value_counts(dropna=False))
print(df['Plano_publico'].isna().sum())

#0     | Não tem plano e/ou não é morador + titular do plano              
#1     | Plano é de instituição pública   
#2     | Plano não é de instituição pública   

Plano_publico
0.0    20870
2.0     2156
1.0     1325
Name: count, dtype: int64
0


In [1501]:
df = df.drop(columns=['I004'])

6. I00401: plano dá direito a consultas
7. I00402: plano dá direito a exames
8. I00403: plano dá direito a internações

In [1502]:
def Cobertura_plano(row):
    # Se não tem plano, retorna 0.0
    if pd.isna(row['I00102']) or row['I00102'] != 1:
        return 0.0

    # Verifica titularidade
    if pd.isna(row['I00103']) or row['I00103'] != 1:
        return 0.0
    if pd.isna(row['C00301']) or pd.isna(row['I001031']) or row['C00301'] != row['I001031']:
        return 0.0

    # Verifica cobertura
    coberturas = [
        row['I00401'] == 1 if pd.notna(row['I00401']) else False,
        row['I00402'] == 1 if pd.notna(row['I00402']) else False,
        row['I00403'] == 1 if pd.notna(row['I00403']) else False
    ]
    total_cobertura = sum(coberturas)

    if total_cobertura == 1:
        return 1.0
    elif total_cobertura == 2:
        return 2.0
    elif total_cobertura == 3:
        return 3.0
    elif total_cobertura == 0:
        return 4.0  # Não cobre nada, apesar de ter plano e titular
    else:
        return np.nan  # Caso de erro

df['Cobertura_plano'] = df.apply(Cobertura_plano, axis=1)

print(df['Cobertura_plano'].value_counts(dropna=False))
print("Valores nulos:", df['Cobertura_plano'].isna().sum())


# 1- se o plano cobre uma das coisas
# 2- se o plano cobre duas das coisas
# 3- se o plano cobre as três coisas

Cobertura_plano
0.0    20870
3.0     3250
2.0      177
1.0       36
4.0       18
Name: count, dtype: int64
Valores nulos: 0


In [1503]:
df = df.drop(columns=['I00102', 'I00401', 'I00402', 'I00403', 'I00103', 'I001031'])

### Preenchendo aqui os atributos J00402, J00404 e J00801

In [1504]:
df.loc[df['J002'] == 2, 'J00402'] = 0
df.loc[df['J002'] == 2, 'J00404'] = 0

In [1505]:
print(df['J00402'].isna().sum())
print(df['J00404'].isna().sum())

0
0


In [1506]:
df.loc[df['J007'] == 2, 'J00801'] = 0
print(df['J00801'].isna().sum())

0


### Exposição tóxica no trabalho

Atributos:
1. M011011: substâncias químicas
2. M011021: barulho intenso (ruído)
3. M011051: lixo (resíduo urbano)
4. M011071: poeira mineral

In [1507]:
def marcar_exposicao_quimica(row):
    trabalho = row['E011']
    exposicao = row['M011011']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_quimica'] = df.apply(marcar_exposicao_quimica, axis=1)

print(df['Exposicao_quimica'].value_counts(dropna=False))
print(df['Exposicao_quimica'].isna().sum())

Exposicao_quimica
2    22934
1     1417
Name: count, dtype: int64
0


In [1508]:
def marcar_exposicao_ruido(row):
    trabalho = row['E011']
    exposicao = row['M011021']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_ruido'] = df.apply(marcar_exposicao_ruido, axis=1)

print(df['Exposicao_ruido'].value_counts(dropna=False))
print(df['Exposicao_ruido'].isna().sum())

Exposicao_ruido
2    22089
1     2262
Name: count, dtype: int64
0


In [1509]:
def marcar_exposicao_lixo(row):
    trabalho = row['E011']
    exposicao = row['M011051']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_lixo'] = df.apply(marcar_exposicao_lixo, axis=1)

print(df['Exposicao_lixo'].value_counts(dropna=False))
print(df['Exposicao_lixo'].isna().sum())

Exposicao_lixo
2    23571
1      780
Name: count, dtype: int64
0


In [1510]:
def marcar_exposicao_minerio(row):
    trabalho = row['E011']
    exposicao = row['M011071']
    
    if trabalho in [1, 2, 3]:  # Pessoa trabalha
        if exposicao == 1:
            return 1  # Exposto
        elif exposicao == 2:
            return 2  # Não exposto
        else:
            return np.nan  # Outro valor ou faltante
    else:
        return 2  # Pessoa não trabalha → marcar como "não exposto"

df['Exposicao_minerio'] = df.apply(marcar_exposicao_minerio, axis=1)

print(df['Exposicao_minerio'].value_counts(dropna=False))
print(df['Exposicao_minerio'].isna().sum())

Exposicao_minerio
2    22985
1     1366
Name: count, dtype: int64
0


In [1511]:
df = df.drop(columns=['M011011'])
df = df.drop(columns=['M011021'])
df = df.drop(columns=['M011051'])
df = df.drop(columns=['M011071'])
df = df.drop(columns=['E011'])

### Dor/desconforto no peito

1. N004: dor ou desconforto no peito ao subir uma ladeira, um lance de escadas ou caminhar rápido no plano
2. N005: dor ou desconforto no peito ao caminhar em lugar plano
3. N008: onde é a dor/desconforto no peito

Outros atributos usados: 

4. G059: usa algum aparelho de auxílio para se locomover? (Se G59=1 , siga N10) - incompleto!

In [1512]:
def map_dor_peito(row):
    if row['G059'] != 2:
        return 'Não sente dor'  # Ignora quem não respondeu 2 em G059 (736 pessoas)

    if row['N004'] == 2 and row['N005'] == 2:
        return 'Não sente dor'

    if row['N004'] == 1 or row['N005'] == 1:
        dor_map = {
            1: 'Acima ou no meio do peito',
            2: 'Abaixo do peito',
            3: 'Braço esquerdo',
            4: 'Outro lugar'
        }
        return dor_map.get(row['N008'], pd.NA)

    return pd.NA  # Se não cair em nenhum caso anterior 

df['Dor_peito'] = df.apply(map_dor_peito, axis=1)

print(df['Dor_peito'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Dor_peito'].isna().sum()}")

Dor_peito
Não sente dor                20996
Acima ou no meio do peito     2572
Abaixo do peito                514
Braço esquerdo                 220
Outro lugar                     49
Name: count, dtype: int64
Valores faltantes: 0


In [1513]:
df = df.drop(columns=['G059', 'N004', 'N005', 'N008'])

### Exercício físico 

1. P034: Nos últimos três meses, praticou algum tipo de exercício físico ou esporte?
2. P035: Quantos dias por semana.
3. P03701: tempo da sessão em horas.
4. P03702: tempo da sessão em minutos.

In [1514]:
print(df['P035'].max())
print(df['P03701'].max())

7.0
10.0


In [1515]:
df['P034'].value_counts()

P034
2.0    16079
1.0     8272
Name: count, dtype: int64

In [1516]:
# Função para calcular o tempo semanal de exercício físico
def calcular_tempo_exercicio(row):
    if row['P034'] == 2:
        return 0
    elif row['P034'] == 1 and row['P035'] > 0:
        horas = row['P03701'] if pd.notnull(row['P03701']) else 0
        minutos = row['P03702'] if pd.notnull(row['P03702']) else 0
        tempo_semanal = row['P035'] * (horas + minutos / 60)
        return min(max(tempo_semanal, 0), 70)
    else:
        return np.nan  # Pode usar 0 ou np.nan dependendo da política de NA

# Aplica o cálculo ao DataFrame
df['TempoExercicioHoras'] = df.apply(calcular_tempo_exercicio, axis=1)

# Função para categorizar em faixas
def categorizar_tempo_personalizado(tempo):
    if tempo == 0:
        return "0h"
    elif tempo <= 10:
        return "até 10h"
    elif tempo <= 20:
        return "10h a 20h"
    elif tempo <= 40:
        return "20h a 40h"
    else:
        return "mais de 40h"

# Aplica a categorização
df['FaixaTempoExercicio'] = df['TempoExercicioHoras'].apply(categorizar_tempo_personalizado)

print(df['FaixaTempoExercicio'].value_counts(dropna=False))
print(f"Valores faltantes: {df['FaixaTempoExercicio'].isna().sum()}")

FaixaTempoExercicio
0h             16079
até 10h         7759
10h a 20h        271
mais de 40h      216
20h a 40h         26
Name: count, dtype: int64
Valores faltantes: 0


In [1517]:
df[['FaixaTempoExercicio']]

,FaixaTempoExercicio
0,até 10h
1,0h
2,até 10h
3,0h
4,até 10h
...,...
24346,0h
24347,0h
24348,0h
24349,0h


In [1518]:
# Mapeamento explícito
mapeamento = {
    "0h": 0,
    "até 10h": 1,
    "10h a 20h": 2,
    "20h a 40h": 3,
    "mais de 40h": 4
}

# Aplicando o mapeamento
df["FaixaTempoExercicio"] = df["FaixaTempoExercicio"].replace(mapeamento).astype(int)

In [1519]:
df[['FaixaTempoExercicio']]

,FaixaTempoExercicio
0,1
1,0
2,1
3,0
4,1
...,...
24346,0
24347,0
24348,0
24349,0


In [1520]:
df = df.drop(columns=['P034', 'P035', 'P03701', 'P03702', 'TempoExercicioHoras'])

### Hábito de fumar / tabagismo

1. P050: Atualmente, o(a) Sr(a) fuma algum produto do tabaco? 
2. P051: E no passado, o(a) Sr(a) fumou algum produto do tabaco diariamente?
3. P052: E no passado, o(a) Sr(a) fumou algum produto do tabaco?
4. P053:  Que idade o(a) Sr(a) tinha quando começou a fumar produto do tabaco diariamente?

    P050 (Fuma atualmente)

1 = Sim, diariamente → usa P053

2 = Sim, menos que diariamente → verifica P051

3 = Não fuma atualmente → verifica P052

    P051 (Fumou diariamente no passado)

1 = Sim → usa P053

2 = Não → considera como não fumante diário

    P052 (Já fumou alguma vez)

1 = Sim → usa P053

2 ou 3 = Não fumou

In [1521]:
def categorizar_inicio_fumo(row):
    p50 = row['P050']
    p51 = row['P051']
    p52 = row['P052']
    p53 = row['P053']  # idade de início

    # Fuma diariamente atualmente
    if p50 == 1:
        idade = p53

    # Fuma menos que diariamente → verifica se já fumou diariamente
    elif p50 == 2:
        if p51 == 1:
            idade = p53
        else:
            return 'Não fuma' 

    # Não fuma atualmente → verifica se já fumou algum dia
    elif p50 == 3:
        if p52 == 1:
            idade = p53
        else:
            return 'Não fuma'

    else:
        return 'Sem informação'  # Resposta inválida ou faltante

    # Categoriza com base na idade de início
    if pd.notna(idade):
        if idade < 18:
            return 'Início antes dos 18'
        elif 18 <= idade <= 30:
            return 'Início entre 18 e 30'
        elif idade > 30:
            return 'Início após os 30'
    return 'Sem informação'  # idade ausente

df['Inicio_tabagismo'] = df.apply(categorizar_inicio_fumo, axis=1)

print(df['Inicio_tabagismo'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Inicio_tabagismo'].isna().sum()}")


Inicio_tabagismo
Não fuma                14010
Início antes dos 18      6739
Início entre 18 e 30     3228
Início após os 30         374
Name: count, dtype: int64
Valores faltantes: 0


In [1522]:
df[["Inicio_tabagismo"]]

,Inicio_tabagismo
0,Início antes dos 18
1,Não fuma
2,Não fuma
3,Início antes dos 18
4,Não fuma
...,...
24346,Não fuma
24347,Não fuma
24348,Início entre 18 e 30
24349,Início após os 30


In [1523]:
# Mapeamento explícito
mapeamento = {
    "Não fuma": 0,
    "Início antes dos 18": 1,
    "Início entre 18 e 30": 2,
    "Início após os 30": 3
}

# Aplicando o mapeamento
df["Inicio_tabagismo"] = df["Inicio_tabagismo"].replace(mapeamento).astype(int)

In [1524]:
df[["Inicio_tabagismo"]]

,Inicio_tabagismo
0,1
1,0
2,0
3,1
4,0
...,...
24346,0
24347,0
24348,2
24349,3


 P59. Há quanto tempo o(a) Sr(a) parou de fumar?

5. P05901 - Anos
6. P05902 - Meses
7. P05903 - Semanas
8. P05904 - Dias

In [1525]:
def tempo_parou_fumar(row):
    # Fumante atual
    if row['P050'] in [1, 2]:
        return 'Fumante atual'

    # Nunca fumou
    if row['P052'] == 3:
        return 'Nunca fumou'

    # Ex-fumante: calcular tempo total em meses
    if row['P052'] in [1, 2]:
        anos = row.get('P05901', 0) or 0
        meses = row.get('P05902', 0) or 0
        semanas = row.get('P05903', 0) or 0
        dias = row.get('P05904', 0) or 0

        total_meses = anos * 12 + meses + (semanas / 4.345) + (dias / 30)

        if pd.isna(total_meses):
            return pd.NA

        if total_meses < 1:
            return 'Parou há <1 mês'
        elif total_meses < 6:
            return '1-6 meses'
        elif total_meses < 12:
            return '6-12 meses'
        elif total_meses < 60:
            return '1-5 anos'
        else:
            return '>5 anos'

    return pd.NA


df['Tempo_parou_fumar'] = df.apply(tempo_parou_fumar, axis=1)

print(df['Tempo_parou_fumar'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Tempo_parou_fumar'].isna().sum()}")

Tempo_parou_fumar
Nunca fumou        13025
>5 anos             7045
Fumante atual       3233
1-5 anos             827
1-6 meses            134
6-12 meses            65
Parou há <1 mês       22
Name: count, dtype: int64
Valores faltantes: 0


In [1526]:
df[["Tempo_parou_fumar"]]

,Tempo_parou_fumar
0,>5 anos
1,Nunca fumou
2,>5 anos
3,>5 anos
4,Nunca fumou
...,...
24346,Nunca fumou
24347,Nunca fumou
24348,Fumante atual
24349,Fumante atual


In [1527]:
# Mapeamento explícito
mapeamento = {
    "Nunca fumou": 0,
    "Fumante atual": 1,
    "Parou há <1 mês": 2,
    "1-6 meses": 3,
    "6-12 meses": 4,
    "1-5 anos": 5,
    ">5 anos": 6
}

# Aplicando o mapeamento
df["Tempo_parou_fumar"] = df["Tempo_parou_fumar"].replace(mapeamento).astype(int)

In [1528]:
df[["Tempo_parou_fumar"]]

,Tempo_parou_fumar
0,6
1,0
2,6
3,6
4,0
...,...
24346,0
24347,0
24348,1
24349,1


9. Cigarros industrializados (P05401)
10. Cigarros de palha ou enrolados (P05404)
11. Cigarros de cravo ou Bali (P05407)
12. Cachimbo (P05410)
13. Charutos ou cigarrilhas (P05413)
14. Narguilé (P05416)
15. Outro (P05419)

In [1529]:
# Dicionário de conversão de respostas (média ponderada semanal)
pesos_fumo = {
    1: 7.0,   # diariamente
    2: 3.0,   # semanalmente
    3: 1.0,   # menos de 1 vez/semana
    4: 0.3,   # menos de 1 por mês
    5: 0.0    # não fuma
}

# Lista dos atributos de frequência
colunas_fumo = ['P05401', 'P05404', 'P05407', 'P05410', 'P05413', 'P05416', 'P05419']

# Substituir as categorias pelos pesos
for col in colunas_fumo:
    df[col + '_peso'] = df[col].map(pesos_fumo)

# Somar os pesos como uma média ponderada de intensidade semanal
df['Intensidade_fumo'] = df[[col + '_peso' for col in colunas_fumo]].sum(axis=1)

# Ajustar para quem nunca fumou (do campo anterior, que chamamos de Inicio_tabagismo)
df.loc[df['Inicio_tabagismo'] == 'Nunca fumou', 'Intensidade_fumo'] = 0.0

# (Opcional) arredondar para facilitar leitura
df['Intensidade_fumo'] = df['Intensidade_fumo'].round(1)

print(df['Intensidade_fumo'].value_counts(dropna=False))
print(f"Valores faltantes: {df['Intensidade_fumo'].isna().sum()}")


Intensidade_fumo
0.0     21118
7.0      2509
14.0      266
3.0       199
8.0        63
7.3        53
1.0        49
10.0       39
0.3        20
6.0        12
2.0         5
21.0        5
4.0         3
0.6         2
3.3         2
21.3        1
9.0         1
0.9         1
7.9         1
14.3        1
7.6         1
Name: count, dtype: int64
Valores faltantes: 0


In [1530]:
df = df.drop(columns=['P050', 'P051', 'P052', 'P053', 'P05401', 'P05404', 'P05407', 'P05410', 'P05413', 'P05416', 'P05419', 'P05901', 'P05902', 'P05903', 'P05904', 
                      'P05401_peso', 'P05404_peso', 'P05407_peso', 'P05410_peso', 'P05413_peso', 'P05416_peso', 'P05419_peso'])

### Atributos sobre doenças cardíacas (módulo Q)

In [1531]:
df.loc[df['Q00101'] == 6, 'Q00201'] = 2 #diagnóstico de hipertensão
df.loc[df['Q02901'] == 6, 'Q03001'] = 2 #diagnóstico de diabetes
df.loc[df['Q05901'] == 6, 'Q060'] = 2 #diagnóstico de colesterol alto

In [1532]:
print(df['Q00201'].isna().sum())
print(df['Q03001'].isna().sum())
print(df['Q060'].isna().sum())

0
0
0


In [1533]:
df = df.drop(columns=['Q00101', 'Q02901', 'Q05901'])    

0 = não tem diagnóstico de doença cardíaca!

In [1534]:
df.loc[df['Q06306'] == 2, 'Q06307'] = 0
df.loc[df['Q06306'] == 2, 'Q06308'] = 0
df.loc[df['Q06306'] == 2, 'Q06309'] = 0
df.loc[df['Q06306'] == 2, 'Q06310'] = 0
df.loc[df['Q06306'] == 2, 'Q06311'] = 0
df.loc[df['Q06306'] == 2, 'Q064'] = 0
df.loc[df['Q06306'] == 2, 'Q06506'] = 0
df.loc[df['Q06306'] == 2, 'Q06507'] = 0
df.loc[df['Q06306'] == 2, 'Q06508'] = 0
df.loc[df['Q06306'] == 2, 'Q06509'] = 0
df.loc[df['Q06306'] == 2, 'Q06601'] = 0
df.loc[df['Q06306'] == 2, 'Q067'] = 0

In [1535]:
print(df['Q06307'].isna().sum())
print(df['Q06308'].isna().sum())
print(df['Q06309'].isna().sum())
print(df['Q06310'].isna().sum())
print(df['Q06311'].isna().sum())
print(df['Q064'].isna().sum())
print(df['Q06506'].isna().sum())
print(df['Q06507'].isna().sum())
print(df['Q06508'].isna().sum())
print(df['Q06509'].isna().sum())
print(df['Q06601'].isna().sum())
print(df['Q067'].isna().sum())

0
0
0
0
0
0
0
0
0
0
0
0


### Preenchendo aqui os atributos H003, H004 e H010

In [1536]:
df.loc[df['H001'].isin([4, 5]), 'H002'] = 0

In [1537]:
print(df['H002'].isna().sum())

0


In [1538]:
df.loc[(df['H002'].isin([1, 0])), 'H003'] = 0

In [1539]:
print(df['H003'].isna().sum())

0


In [1540]:
df.loc[df['H003'] == 0, 'H004'] = 0

In [1541]:
print(df['H004'].isna().sum())

0


In [1542]:
df.loc[df['H004'] != 1, 'H010'] = 0

In [1543]:
print(df['H010'].isna().sum())

0


### Hábito de beber (alcoolismo) - incompleto (só checar)

1. P027: Com que frequência costuma consumir alguma bebida alcoólica?
2. P02801: Quantos dias por semana costuma consumir alguma bebida alcoólica?
3. P029:  Em geral, no dia que bebe, quantas doses de bebida alcoólica consome?

In [1544]:
print(df['P02801'].max()) # = 7 dias
print(df['P029'].max()) # = 60 doses

# → Máximo teórico de doses por semana = 7 * 60 = 420 doses

7.0
60.0


In [1545]:
def calcular_doses_semanais(row):
    freq = row['P027']
    dias_semana = row['P02801']
    doses_por_dia = row['P029']

    if freq == 1:
        return 0  # Não bebe nunca
    elif freq == 2:
        return doses_por_dia if pd.notna(doses_por_dia) else pd.NA
    elif freq == 3:
        if pd.notna(dias_semana) and pd.notna(doses_por_dia):
            return dias_semana * doses_por_dia
        else:
            return pd.NA
    return pd.NA

def categorizar_doses(doses):
    if pd.isna(doses):
        return pd.NA
    elif doses == 0:
        return '0 doses'
    elif doses <= 5:
        return '1 a 5 doses'
    elif doses <= 14:
        return '6 a 14 doses'
    elif doses <= 30:
        return '15 a 30 doses'
    elif doses <= 60:
        return '31 a 60 doses'
    else:
        return '61+ doses'

df['Doses_semanais'] = df.apply(calcular_doses_semanais, axis=1)
df['Quant_doses_alcool_semanal'] = df['Doses_semanais'].apply(categorizar_doses)


print(df['Quant_doses_alcool_semanal'].value_counts(dropna=False))
print("Valores faltantes:", df['Quant_doses_alcool_semanal'].isna().sum())


Quant_doses_alcool_semanal
0 doses          17191
1 a 5 doses       4070
6 a 14 doses      2008
15 a 30 doses      785
31 a 60 doses      218
61+ doses           79
Name: count, dtype: int64
Valores faltantes: 0


In [1546]:
df[["Quant_doses_alcool_semanal"]]

,Quant_doses_alcool_semanal
0,0 doses
1,0 doses
2,15 a 30 doses
3,6 a 14 doses
4,0 doses
...,...
24346,6 a 14 doses
24347,0 doses
24348,1 a 5 doses
24349,15 a 30 doses


In [1547]:
mapeamento = {
    "0 doses": 0,
    "1 a 5 doses": 1,
    "6 a 14 doses": 2,
    "15 a 30 doses": 3,
    "31 a 60 doses": 4,
    "61+ doses": 5
}

# Aplicando o mapeamento
df["Quant_doses_alcool_semanal"] = df["Quant_doses_alcool_semanal"].replace(mapeamento).astype(int)

In [1548]:
df[["Quant_doses_alcool_semanal"]]

,Quant_doses_alcool_semanal
0,0
1,0
2,3
3,2
4,0
...,...
24346,2
24347,0
24348,1
24349,3


In [1549]:
df = df.drop(columns=['P027','P02801', 'P029'])

### Peso e altura

1. P00103: Peso - Informado (em kg)
2. P00104: Peso - Final (em kg)
3. P00403: Altura - Informada (em cm)
4. P00404: Altura - Final (em cm)

In [1550]:
print(df['P00103'].isna().sum())
print(df['P00104'].isna().sum())

2172
27


In [1551]:
print(df['P00403'].isna().sum())
print(df['P00404'].isna().sum())

5601
27


In [1552]:
faltantes_iguais = df[(df['P00104'].isna()) & (df['P00103'].notna())][['P00104', 'P00103']]
faltantes_iguais

,P00104,P00103


In [1553]:
faltantes_p00104 = df[df['P00104'].isna()][['P00103', 'P00104']]
faltantes_p00104

,P00103,P00104
479,NaN,NaN
1722,NaN,NaN
1749,NaN,NaN
1794,NaN,NaN
5239,NaN,NaN
5330,NaN,NaN
6533,NaN,NaN
7441,NaN,NaN
7941,NaN,NaN
9154,NaN,NaN


In [1554]:
faltantes_iguais = df[(df['P00404'].isna()) & (df['P00403'].notna())][['P00404', 'P00403']]
faltantes_iguais

,P00404,P00403


In [1555]:
faltantes_p00404 = df[df['P00404'].isna()][['P00403', 'P00404']]
faltantes_p00404

,P00403,P00404
479,NaN,NaN
1722,NaN,NaN
1749,NaN,NaN
1794,NaN,NaN
5239,NaN,NaN
5330,NaN,NaN
6533,NaN,NaN
7441,NaN,NaN
7941,NaN,NaN
9154,NaN,NaN


In [1556]:
faltantes = df['P00104'].isna().equals(df['P00404'].isna())
faltantes

True

In [1557]:
df = df.drop(columns=['P00103', 'P00403'])

### Lalala

In [1558]:
print("M00203 faltantes:", df['M00203'].isna().sum())
print(df['M00203'].value_counts(dropna=False)) #ver isso aqui (incompleto)

M00203 faltantes: 3
M00203
1.0    23962
2.0      383
NaN        3
9.0        3
Name: count, dtype: int64


In [1559]:
df = df[~(df['M00203'].isna() | (df['M00203'] == 9))]

In [1560]:
print(df['M00203'].value_counts(dropna=False))

M00203
1.0    23962
2.0      383
Name: count, dtype: int64


In [1561]:
df.to_csv('../Bases/Criados/base_preenchida.csv', index=False)

In [1562]:
df.columns[df.isnull().any()]

Index(['F001021', 'F007021', 'F008021', 'VDF00102', 'P00104', 'P00404'], dtype='object')

In [1563]:
(df.isnull().sum() > 0).sum()

np.int64(6)